In [4]:
import csv
from IPython.display import display, Latex

alphabet = [chr(i+ord('a')) for i in range(26)]
alphabet.append('.')
letter2idx = {c:i for i,c in enumerate(alphabet)}

T = []
with open('transitions.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        T.append(row)

print('Example')
## p(x_t = 'u' | x_{t-1} = 'q')
display(Latex(r"$p(x_t = \text{'u'} | x_{t-1} = \text{'q'})$"))
print(T[letter2idx['q']][letter2idx['u']])
display(Latex(r"$p(x_t | x_{t-1} = \text{'a'})$"))
for c,p in zip(alphabet,T[letter2idx['a']]):
    print(c,p)    

Example


<IPython.core.display.Latex object>

0.9949749


<IPython.core.display.Latex object>

a 0.0002835
b 0.0228302
c 0.0369041
d 0.0426290
e 0.0012216
f 0.0075739
g 0.0171385
h 0.0014659
i 0.0372661
j 0.0002353
k 0.0110124
l 0.0778259
m 0.0260757
n 0.2145354
o 0.0005459
p 0.0195213
q 0.0001749
r 0.1104770
s 0.0934290
t 0.1317960
u 0.0098029
v 0.0306574
w 0.0088799
x 0.0009562
y 0.0233701
z 0.0018701
. 0.0715219


### TASK 1

In [143]:
import numpy as np
from random import randint
N=10
letters=[]
letters.append('.');
for i in range (0,N):
    prev=letters[i]
    prev_id=letter2idx[prev]
    column=T[:][prev_id]
    probs=np.array([float(j) for j in column])
    probs /= probs.sum()
    c=np.random.choice(alphabet,p=probs)
    letters.append(c)
str=""
for j in letters[1:]:
    str+=j
print(str)  

is.t.ofome


## TASK2

In [162]:
from numpy import linalg as LA
test_strings = ['th__br__n.f_x.', '_u_st__n_.to_be._nsw_r__','i__at_._a_h_n_._e_r_i_g','q___t.___z._____t.__.___.__.']
#Convert T to float array.
T=np.array(T).astype(np.float)

def estimate_interval(low,high,previ,nexti,charList):
        length=high-low+1
        if length==0:
            return charList
        
        row=T[:][previ]
        if(high==len(charList)-1):
            column=1
        else :
            column=[t[nexti] for t in T]
        
        v=row
        if length>1:
            v=np.dot(v,LA.matrix_power(T, length-1))
        
        w=list(v*column)
        final=np.random.choice(alphabet,p=w/np.sum(w))
        charList[high]=final
        if length==1:
            return charList
        else:
            return estimate_interval(low,high-1,previ,letter2idx[final],charList)
 

def fill_missing_letters(string):    
    N=len(string)
    unknown=[]
    sum_terms=[]
    for ch in range (0,N):
        current=string[ch]
        if current=='?' or current=='_':
            unknown.append(ch)


    for i in range(0,len(unknown)):
        temp=[]
        if i==0:
            temp.append(unknown[i])
            sum_terms.append(temp)
        elif unknown[i]-unknown[i-1]==1:
            sum_terms[len(sum_terms)-1].append(unknown[i])
        else :
            temp.append(unknown[i])
            sum_terms.append(temp)


        
    result=list(list(string))
    for i in range(0,len(sum_terms)):
        low=sum_terms[i][0]
        up=sum_terms[i][len(sum_terms[i])-1]
        prevId=letter2idx[string[low-1]]
        if(up!=len(string)-1):
            nextId=letter2idx[string[up+1]]
        else :
            nextId=-1

        result=estimate_interval(low,up,prevId,nextId,result)

    return result

for i in test_strings:
    result='.'+i
    result=fill_missing_letters(result)
    print(''.join(result[1:len(result)]))




the.bru.n.fex.
.u.st.ane.to.be.answorat
is.at..wathend.beer.ing
qut.t.rerz.t.w..t.be.bre.m..


## TASK3

In [153]:
from numpy import linalg as LA
test_strings = ['th__br__n.f_x.', '_u_st__n_.to_be._nsw_r__','i__at_._a_h_n_._e_r_i_g','q___t.___z._____t.__.___.__.']
#Convert T to float array.
T=np.array(T).astype(np.float)

def estimate_interval(low,high,previ,nexti,charList):
        length=high-low+1
        if length==0:
            return charList
        
        row=T[:][previ]
        if(high==len(charList)-1):
            column=1
        else :
            column=[t[nexti] for t in T]
        
        v=row
        if length>1:
            v=np.dot(v,LA.matrix_power(T, length-1))
        
        w=list(v*column)
        final=w.index(np.max(w))
        charList[high]=alphabet[final]
        if length==1:
            return charList
        else:
            return estimate_interval(low,high-1,previ,letter2idx[alphabet[final]],charList)
 

def fill_missing_letters(string):    
    N=len(string)
    unknown=[]
    sum_terms=[]
    for ch in range (0,N):
        current=string[ch]
        if current=='?' or current=='_':
            unknown.append(ch)


    for i in range(0,len(unknown)):
        temp=[]
        if i==0:
            temp.append(unknown[i])
            sum_terms.append(temp)
        elif unknown[i]-unknown[i-1]==1:
            sum_terms[len(sum_terms)-1].append(unknown[i])
        else :
            temp.append(unknown[i])
            sum_terms.append(temp)


        
    result=list(list(string))
    for i in range(0,len(sum_terms)):
        low=sum_terms[i][0]
        up=sum_terms[i][len(sum_terms[i])-1]
        prevId=letter2idx[string[low-1]]
        if(up!=len(string)-1):
            nextId=letter2idx[string[up+1]]
        else :
            nextId=-1

        result=estimate_interval(low,up,prevId,nextId,result)

    return result

for i in test_strings:
    result='.'+i
    result=fill_missing_letters(result)
    print(''.join(result[1:len(result)]))



    

the.br.an.fex.
oursthand.to.be.answere.
in.ath.wathend.he.r.ing
qus.t.herz.athe.t.he.the.he.
